# LLM Testing

## Setup

In [ ]:
# Import
import pandas as pd
import requests
from openai import OpenAI

# API keys
ALPHA_VANTAGE_API_KEY = ""
OPEN_ROUTER_API_KEY = ""

## Alpha Vantage API

In [ ]:
def get_company_overview(api_key, ticker):
    """
    Fetches the company overview from Alpha Vantage using function=OVERVIEW.
    Returns a Pandas DataFrame.
    """
    url = "https://www.alphavantage.co/query"
    params = {
        "function": "OVERVIEW",
        "symbol": ticker,
        "apikey": api_key
    }

    response = requests.get(url, params=params)
    data = response.json()

    if "Symbol" not in data:
        print(f"Error fetching company overview: {data}")
        return None

    # Convert JSON dictionary to a Pandas DataFrame
    overview_df = pd.DataFrame.from_dict(data, orient='index', columns=['Value'])
    overview_df.reset_index(inplace=True)
    overview_df.columns = ['Field', 'Value']

    return overview_df

def fetch_and_display_overview(api_key):
    """
    Prompts user for a stock ticker, fetches the company overview using Alpha Vantage API,
    and returns it as a Pandas DataFrame.
    """
    ticker = input("Enter the company ticker symbol (e.g., AAPL, TSLA): ").upper()
    overview_df = get_company_overview(api_key, ticker)

    if overview_df is not None:
        print("\n=== Company Overview ===")
        print(overview_df)
    else:
        print("Failed to retrieve company overview.")

    return overview_df  # Returns the DataFrame for further use if needed

# Example usage
company_overview_df = fetch_and_display_overview(ALPHA_VANTAGE_API_KEY)



=== Company Overview ===
                         Field  \
0                       Symbol   
1                    AssetType   
2                         Name   
3                  Description   
4                          CIK   
5                     Exchange   
6                     Currency   
7                      Country   
8                       Sector   
9                     Industry   
10                     Address   
11                OfficialSite   
12               FiscalYearEnd   
13               LatestQuarter   
14        MarketCapitalization   
15                      EBITDA   
16                     PERatio   
17                    PEGRatio   
18                   BookValue   
19            DividendPerShare   
20               DividendYield   
21                         EPS   
22          RevenuePerShareTTM   
23                ProfitMargin   
24          OperatingMarginTTM   
25           ReturnOnAssetsTTM   
26           ReturnOnEquityTTM   
27                  Re

## Openrouter

### Deepseek R1 (free)

In [ ]:
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=OPEN_ROUTER_API_KEY,
)

completion = client.chat.completions.create(
  extra_headers={
    "HTTP-Referer": "<YOUR_SITE_URL>", # Optional. Site URL for rankings on openrouter.ai.
    "X-Title": "<YOUR_SITE_NAME>", # Optional. Site title for rankings on openrouter.ai.
  },
  extra_body={},
  model="deepseek/deepseek-r1:free",
  messages=[
    {
      "role": "user",
      "content": "What is the meaning of life?"
    }
  ]
)
print(completion.choices[0].message.content)

The meaning of life is a deeply personal and philosophically complex question with no universally agreed-upon answer. Here’s a synthesis of perspectives:

1. **Philosophical Views**: 
   - **Existentialism** posits that life has no inherent meaning, and individuals must create their own purpose through choices and actions (e.g., Sartre, Camus).
   - **Virtue Ethics** (Aristotle) suggests meaning arises from pursuing excellence, virtue, and flourishing (*eudaimonia*).
   - **Absurdism** acknowledges the conflict between humanity’s search for meaning and the universe’s indifference, advocating for embracing life despite its lack of inherent purpose.

2. **Religious/Spiritual Perspectives**:
   - Many religions define meaning through a divine purpose, such as serving a higher power (Christianity, Islam), achieving enlightenment (Buddhism), or aligning with cosmic order (Hinduism).

3. **Scientific and Evolutionary Angles**:
   - Biology frames life’s purpose in terms of survival and repro

### Llama 3.3 70B Instruct (free)

In [ ]:
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=OPEN_ROUTER_API_KEY,
)

completion = client.chat.completions.create(
  extra_headers={
    "HTTP-Referer": "<YOUR_SITE_URL>", # Optional. Site URL for rankings on openrouter.ai.
    "X-Title": "<YOUR_SITE_NAME>", # Optional. Site title for rankings on openrouter.ai.
  },
  extra_body={},
  model="meta-llama/llama-3.3-70b-instruct:free",
  messages=[
      {
          "role": "system",
          "content": (
              "You are a knowledgeable financial assistant specializing in analyzing companies based on real-time "
              "financial data and news articles. You assist the user by answering questions about a specific company "
              "using the provided company overview and retrieved news articles."
              "\n\n=== Company Overview ===\n"
              "{company_overview}\n\n"
              "Use this information as context when responding to user queries. Keep responses concise, factual, "
              "and focused on financial analysis. If there is missing information, acknowledge it instead of speculating."
          )
      },
      {
          "role": "assistant",
          "content": (
              "Based on the provided company overview and recent news, here is an analysis:\n\n"
              "[Example Response]:\n"
              "- **Company Performance**: The company's market capitalization is currently {market_cap}, indicating its size within the sector.\n"
              "- **Recent Trends**: Recent news suggests that {key_news_event} has impacted stock movement.\n"
              "- **Industry Standing**: Compared to competitors, {company_name} has shown {relative_performance}.\n\n"
              "Would you like insights on a specific financial metric?"
          )
      },
      {
          "role": "user",
          "content": "{user_query}"
      }
  ]
)  
print(completion.choices[0].message.content)

The meaning of life is a question that has puzzled philosophers, theologians, scientists, and many others for centuries. It's a complex and deeply personal question that can have different answers depending on one's cultural, religious, and individual perspectives. Here's a brief exploration of some possible approaches to understanding the meaning of life:

1. **Philosophical perspectives**: Philosophers have offered various interpretations, including:
	* **Hedonism**: Life's meaning is found in the pursuit of pleasure and happiness.
	* **Epicureanism**: Life's meaning is found in the absence of physical pain and mental distress.
	* **Existentialism**: Life has no inherent meaning; individuals must create their own purpose.
	* **Humanism**: Life's meaning is found in the pursuit of knowledge, beauty, and human flourishing.
2. **Religious and spiritual perspectives**: Many religious traditions offer answers to the question of life's meaning, such as:
	* **Christianity**: Life's meaning 

## Prompt Engineering

In [7]:
def get_company_overview(api_key, ticker):
    """
    Fetches the company overview from Alpha Vantage using function=OVERVIEW.
    Returns a string formatted for LLM input.
    """
    url = "https://www.alphavantage.co/query"
    params = {
        "function": "OVERVIEW",
        "symbol": ticker,
        "apikey": api_key
    }

    response = requests.get(url, params=params)
    data = response.json()

    if "Symbol" not in data:
        return f"Error fetching company overview: {data}"

    # Convert JSON dictionary to formatted text
    overview_text = "\n".join([f"{key}: {val}" for key, val in data.items()])

    return overview_text  # Returns as a text string for the LLM


def fetch_company_and_query_llm():
    """
    Prompts user for a stock ticker, fetches the company overview, 
    and allows the user to ask financial questions.
    """
    # Get the stock ticker from the user
    ticker = input("Enter the company ticker symbol (e.g., AAPL, TSLA): ").upper()

    print("\nFetching company overview...\n")
    company_overview_text = get_company_overview(ALPHA_VANTAGE_API_KEY, ticker)

    print("\n=== Company Overview ===")
    print(company_overview_text)  # Print for user reference

    # Get user query
    user_query = input("\nNow ask a financial question about this company: ")

    # Set up OpenRouter API client
    client = OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key=OPEN_ROUTER_API_KEY,
    )

    # Construct the prompt
    messages = [
        {
            "role": "system",
            "content": (
                "You are a knowledgeable financial assistant specializing in analyzing companies based on real-time "
                "financial data and market news. You assist the user by answering questions about a specific company "
                "using the provided company overview.\n\n"
                "=== Company Overview ===\n"
                f"{company_overview_text}\n\n"
                "Use this information as context when responding to user queries. Keep responses concise, factual, "
                "and focused on financial analysis. If there is missing information, acknowledge it instead of speculating."
            )
        },
        {
            "role": "assistant",
            "content": (
                "Based on the provided company overview, here is an analysis:\n\n"
                "- **Company Performance**: The company's market capitalization is {market_cap}, indicating its size in the industry.\n"
                "- **Recent Trends**: Market conditions and news suggest {key_news_event} has impacted stock performance.\n"
                "- **Industry Standing**: Compared to competitors, {company_name} has shown {relative_performance}.\n\n"
                "Would you like insights on a specific financial metric?"
            )
        },
        {
            "role": "user",
            "content": user_query
        }
    ]

    # Call the LLM API
    completion = client.chat.completions.create(
        model="meta-llama/llama-3.3-70b-instruct:free",
        messages=messages
    )

    # Print the AI's response
    print("\n=== AI Response ===\n")
    print(completion.choices[0].message.content)


# Run the interactive assistant
if __name__ == "__main__":
    fetch_company_and_query_llm()



Fetching company overview...


=== Company Overview ===
Symbol: AAPL
AssetType: Common Stock
Name: Apple Inc
Description: Apple Inc. is an American multinational technology company that specializes in consumer electronics, computer software, and online services. Apple is the world's largest technology company by revenue (totalling $274.5 billion in 2020) and, since January 2021, the world's most valuable company. As of 2021, Apple is the world's fourth-largest PC vendor by unit sales, and fourth-largest smartphone manufacturer. It is one of the Big Five American information technology companies, along with Amazon, Google, Microsoft, and Facebook.
CIK: 320193
Exchange: NASDAQ
Currency: USD
Country: USA
Sector: TECHNOLOGY
Industry: ELECTRONIC COMPUTERS
Address: ONE INFINITE LOOP, CUPERTINO, CA, US
OfficialSite: https://www.apple.com
FiscalYearEnd: September
LatestQuarter: 2024-12-31
MarketCapitalization: 3544164073000
EBITDA: 137352004000
PERatio: 37.75
PEGRatio: 2.185
BookValue: 4.438
